## Import Libraries

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

## Hyperparameters

In [2]:
RANDOM_SEED = 171

N_FOLDS = 10
N_EPOCHS = 100
BATCH_SIZE = 64

OPTIMIZER = 'adam'
LOSS_FUNCTION = 'mean_squared_error'

HIDDEN_LAYER_UNITS = 256
OUTPUT_LAYER_UNITS = 1
HIDDEN_LAYER_ACTIVATION = 'relu'
OUTPUT_LAYER_ACTIVATION = 'sigmoid'

## Import Data

In [3]:
# Define a dictionary to store all the datasets 
season_statistics = {} 

for year in range(1994, 2024): 
    # Import the CSV file for the corresponding year as a pandas df 
    df = pd.read_csv(f"./data/{year}.csv") 
    
    # Add the dataframe to our dictionary of dataframes 
    season_statistics[year] = df

## Preprocess Data

In [4]:
# Concatenate all season data
df = pd.concat(season_statistics.values(), ignore_index=True)

# Define Categorical and Numerical Columns for Pre-Processing
selected_features = df.columns.drop(['Champion', 'Arena', 'Team', 'Year', 'G'])
# selected_features = ['W%', 'FG%', 'FGA', '3P%', 'FT%', 'ORB', 'DRB', 'AST', 'STL', 'BLK', 
#                      'PPG', 'OPPG', 'Age', 'ORtg', 'DRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'eFG%', 'Attend./G']

# Create a Data PreProcessor
preprocessor = ColumnTransformer(transformers = [('num', StandardScaler(), selected_features)])

# Separate the Predictors and Response
X = df[selected_features]
y = df['Champion'].values

# Pre-Process the Input Data
X = preprocessor.fit_transform(X)

## Train Model

In [5]:
# Define the K-Fold Cross Validation Sets
kf = KFold(n_splits = N_FOLDS, shuffle = True, random_state = RANDOM_SEED)

# Create a List for each Model and its Accuracy
model_history, mse_history = [], []

# Perform K-Fold Cross Validation
for train_index, test_index in kf.split(X):
    
    # Split into Training and Testing Datasets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Build the Neural Network Model
    model = Sequential([
        Dense(HIDDEN_LAYER_UNITS, activation = HIDDEN_LAYER_ACTIVATION, input_shape=(X_train.shape[1],)),
        Dense(HIDDEN_LAYER_UNITS, activation = HIDDEN_LAYER_ACTIVATION),
        Dense(HIDDEN_LAYER_UNITS, activation = HIDDEN_LAYER_ACTIVATION),
        Dense(OUTPUT_LAYER_UNITS, activation = OUTPUT_LAYER_ACTIVATION)
    ])
    model.compile(optimizer = OPTIMIZER, loss = LOSS_FUNCTION, metrics = 'mse')
    model.fit(X_train, y_train, epochs = N_EPOCHS, batch_size = BATCH_SIZE, verbose = 1)
    
    # Predict and evaluate
    y_pred = model.predict(X_test).flatten()
    y_pred, y_test = (y_pred * 100), (y_test * 100)
    mse = mean_squared_error(y_test, y_pred)
    
    # Append the Model and MSE to the respective lists
    model_history.append(model)
    mse_history.append(mse)

Epoch 1/100
13/13 [==============================] - 1s 3ms/step - loss: 0.0583 - mse: 0.0583
Epoch 2/100
13/13 [==============================] - 0s 3ms/step - loss: 0.0413 - mse: 0.0413
Epoch 3/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0290 - mse: 0.0290
Epoch 4/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0220 - mse: 0.0220
Epoch 5/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0186 - mse: 0.0186
Epoch 6/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0156 - mse: 0.0156
Epoch 7/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0136 - mse: 0.0136
Epoch 8/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0111 - mse: 0.0111
Epoch 9/100
13/13 [==============================] - 0s 3ms/step - loss: 0.0104 - mse: 0.0104
Epoch 10/100
13/13 [==============================] - 0s 3ms/step - loss: 0.0090 - mse: 0.0090
Epoch 11/100
13/13 [==============================] - 0s 2m

13/13 [==============================] - 0s 2ms/step - loss: 5.4695e-04 - mse: 5.4695e-04
Epoch 84/100
13/13 [==============================] - 0s 2ms/step - loss: 4.1345e-04 - mse: 4.1345e-04
Epoch 85/100
13/13 [==============================] - 0s 2ms/step - loss: 3.9058e-04 - mse: 3.9058e-04
Epoch 86/100
13/13 [==============================] - 0s 2ms/step - loss: 3.6813e-04 - mse: 3.6813e-04
Epoch 87/100
13/13 [==============================] - 0s 2ms/step - loss: 2.7664e-04 - mse: 2.7664e-04
Epoch 88/100
13/13 [==============================] - 0s 2ms/step - loss: 1.9591e-04 - mse: 1.9591e-04
Epoch 89/100
13/13 [==============================] - 0s 4ms/step - loss: 2.0945e-04 - mse: 2.0945e-04
Epoch 90/100
13/13 [==============================] - 0s 3ms/step - loss: 2.7484e-04 - mse: 2.7484e-04
Epoch 91/100
13/13 [==============================] - 0s 3ms/step - loss: 2.3942e-04 - mse: 2.3942e-04
Epoch 92/100
13/13 [==============================] - 0s 8ms/step - loss: 1.8639e-04 -

13/13 [==============================] - 0s 2ms/step - loss: 4.0113e-04 - mse: 4.0113e-04
Epoch 66/100
13/13 [==============================] - 0s 2ms/step - loss: 3.6445e-04 - mse: 3.6445e-04
Epoch 67/100
13/13 [==============================] - 0s 2ms/step - loss: 3.3096e-04 - mse: 3.3096e-04
Epoch 68/100
13/13 [==============================] - 0s 2ms/step - loss: 3.0169e-04 - mse: 3.0169e-04
Epoch 69/100
13/13 [==============================] - 0s 2ms/step - loss: 2.8435e-04 - mse: 2.8435e-04
Epoch 70/100
13/13 [==============================] - 0s 2ms/step - loss: 2.7252e-04 - mse: 2.7252e-04
Epoch 71/100
13/13 [==============================] - 0s 2ms/step - loss: 2.4580e-04 - mse: 2.4580e-04
Epoch 72/100
13/13 [==============================] - 0s 2ms/step - loss: 2.3555e-04 - mse: 2.3555e-04
Epoch 73/100
13/13 [==============================] - 0s 2ms/step - loss: 2.2395e-04 - mse: 2.2395e-04
Epoch 74/100
13/13 [==============================] - 0s 2ms/step - loss: 2.1681e-04 -

13/13 [==============================] - 0s 3ms/step - loss: 0.0016 - mse: 0.0016
Epoch 48/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0017 - mse: 0.0017
Epoch 49/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0018 - mse: 0.0018
Epoch 50/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0016 - mse: 0.0016
Epoch 51/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0017 - mse: 0.0017
Epoch 52/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0015 - mse: 0.0015
Epoch 53/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0013 - mse: 0.0013    
Epoch 54/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0014 - mse: 0.0014
Epoch 55/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0013 - mse: 0.0013
Epoch 56/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0012 - mse: 0.0012    
Epoch 57/100
13/13 [==============================] - 0

13/13 [==============================] - 0s 3ms/step - loss: 0.0023 - mse: 0.0023    
Epoch 31/100
13/13 [==============================] - 0s 3ms/step - loss: 0.0024 - mse: 0.0024
Epoch 32/100
13/13 [==============================] - 0s 4ms/step - loss: 0.0023 - mse: 0.0023
Epoch 33/100
13/13 [==============================] - 0s 6ms/step - loss: 0.0022 - mse: 0.0022
Epoch 34/100
13/13 [==============================] - 0s 4ms/step - loss: 0.0018 - mse: 0.0018    
Epoch 35/100
13/13 [==============================] - 0s 3ms/step - loss: 0.0017 - mse: 0.0017
Epoch 36/100
13/13 [==============================] - 0s 8ms/step - loss: 0.0018 - mse: 0.0018
Epoch 37/100
13/13 [==============================] - 0s 5ms/step - loss: 0.0016 - mse: 0.0016
Epoch 38/100
13/13 [==============================] - 0s 11ms/step - loss: 0.0013 - mse: 0.0013
Epoch 39/100
13/13 [==============================] - 0s 4ms/step - loss: 0.0013 - mse: 0.0013    
Epoch 40/100
13/13 [==============================

...

13/13 [==============================] - 0s 2ms/step - loss: 0.0016 - mse: 0.0016    
Epoch 45/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0016 - mse: 0.0016
Epoch 46/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0013 - mse: 0.0013
Epoch 47/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0011 - mse: 0.0011
Epoch 48/100
13/13 [==============================] - 0s 2ms/step - loss: 9.4991e-04 - mse: 9.4991e-04
Epoch 49/100
13/13 [==============================] - 0s 2ms/step - loss: 8.2197e-04 - mse: 8.2197e-04
Epoch 50/100
13/13 [==============================] - 0s 2ms/step - loss: 8.1665e-04 - mse: 8.1665e-04
Epoch 51/100
13/13 [==============================] - 0s 2ms/step - loss: 7.0675e-04 - mse: 7.0675e-04
Epoch 52/100
13/13 [==============================] - 0s 2ms/step - loss: 7.7163e-04 - mse: 7.7163e-04
Epoch 53/100
13/13 [==============================] - 0s 2ms/step - loss: 6.6493e-04 - mse: 6.6493e-04
Epoch 54/10

13/13 [==============================] - 0s 2ms/step - loss: 0.0023 - mse: 0.0023    
Epoch 26/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0021 - mse: 0.0021
Epoch 27/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0020 - mse: 0.0020
Epoch 28/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0020 - mse: 0.0020
Epoch 29/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0025 - mse: 0.0025
Epoch 30/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0030 - mse: 0.0030
Epoch 31/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0026 - mse: 0.0026
Epoch 32/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0023 - mse: 0.0023
Epoch 33/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0020 - mse: 0.0020
Epoch 34/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0017 - mse: 0.0017    
Epoch 35/100
13/13 [==============================] - 0

13/13 [==============================] - 0s 2ms/step - loss: 0.0136 - mse: 0.0136
Epoch 7/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0109 - mse: 0.0109
Epoch 8/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0102 - mse: 0.0102
Epoch 9/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0080 - mse: 0.0080
Epoch 10/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0064 - mse: 0.0064
Epoch 11/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0058 - mse: 0.0058
Epoch 12/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0065 - mse: 0.0065
Epoch 13/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0049 - mse: 0.0049
Epoch 14/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0045 - mse: 0.0045
Epoch 15/100
13/13 [==============================] - 0s 2ms/step - loss: 0.0039 - mse: 0.0039
Epoch 16/100
13/13 [==============================] - 0s 2ms/step 

13/13 [==============================] - 0s 2ms/step - loss: 3.1328e-04 - mse: 3.1328e-04
Epoch 89/100
13/13 [==============================] - 0s 2ms/step - loss: 2.7229e-04 - mse: 2.7229e-04
Epoch 90/100
13/13 [==============================] - 0s 2ms/step - loss: 2.1366e-04 - mse: 2.1366e-04
Epoch 91/100
13/13 [==============================] - 0s 2ms/step - loss: 2.6424e-04 - mse: 2.6424e-04
Epoch 92/100
13/13 [==============================] - 0s 2ms/step - loss: 3.0213e-04 - mse: 3.0213e-04
Epoch 93/100
13/13 [==============================] - 0s 2ms/step - loss: 3.4722e-04 - mse: 3.4722e-04
Epoch 94/100
13/13 [==============================] - 0s 2ms/step - loss: 2.6076e-04 - mse: 2.6076e-04
Epoch 95/100
13/13 [==============================] - 0s 2ms/step - loss: 2.1922e-04 - mse: 2.1922e-04
Epoch 96/100
13/13 [==============================] - 0s 2ms/step - loss: 1.9107e-04 - mse: 1.9107e-04
Epoch 97/100
13/13 [==============================] - 0s 2ms/step - loss: 1.7247e-04 -

13/13 [==============================] - 0s 2ms/step - loss: 6.3651e-04 - mse: 6.3651e-04
Epoch 72/100
13/13 [==============================] - 0s 2ms/step - loss: 7.0310e-04 - mse: 7.0310e-04
Epoch 73/100
13/13 [==============================] - 0s 2ms/step - loss: 6.0687e-04 - mse: 6.0687e-04
Epoch 74/100
13/13 [==============================] - 0s 2ms/step - loss: 6.4012e-04 - mse: 6.4012e-04
Epoch 75/100
13/13 [==============================] - 0s 2ms/step - loss: 6.9005e-04 - mse: 6.9005e-04
Epoch 76/100
13/13 [==============================] - 0s 2ms/step - loss: 8.4208e-04 - mse: 8.4208e-04
Epoch 77/100
13/13 [==============================] - 0s 2ms/step - loss: 9.0271e-04 - mse: 9.0271e-04
Epoch 78/100
13/13 [==============================] - 0s 2ms/step - loss: 8.0740e-04 - mse: 8.0740e-04
Epoch 79/100
13/13 [==============================] - 0s 2ms/step - loss: 9.4287e-04 - mse: 9.4287e-04
Epoch 80/100
13/13 [==============================] - 0s 2ms/step - loss: 8.7663e-04 -

## Evaluate Model

In [6]:
# Print Performance Metrics
print("Average MSE:", np.mean(mse_history), '\n')
for i in range(len(model_history)):
    print(f"Model #{i + 1}, MSE: {mse_history[i]}")
    
# Get the Best Model
best_model_idx = mse_history.index(min(mse_history))
best_model = model_history[best_model_idx]

# Perform Accuracy Test
accuracy = 0
print("\n\nChampionship Prediction (Historic)\n----------------------------------\n")
for year, df in season_statistics.items():
    
    # Get the Actual Champion
    real_champion = df[df['Champion'] == 1]['Team'].values[0]
    
    # Get the Model Prediction
    X = preprocessor.fit_transform(df[selected_features])
    y = best_model.predict(X)
    pred_champion = df.iloc[np.argmax(y)]['Team']
    
    # Evaluate Results
    accuracy += int(real_champion == pred_champion)
    print(f"{year} NBA Season")
    print(f"\tActual Champion: {real_champion}")
    print(f"\tPredicted Champion: {pred_champion}")
    print()
    
print(f"Model Accuracy: {(accuracy / len(season_statistics) * 100):.2f}% ({accuracy}/{len(season_statistics)})")

Average MSE: 369.3067591714424 

Model #1, MSE: 204.1024678709961
Model #2, MSE: 318.6077253501351
Model #3, MSE: 329.34009092778336
Model #4, MSE: 213.5656674791296
Model #5, MSE: 305.17013560175207
Model #6, MSE: 544.9753463045398
Model #7, MSE: 340.4123226672695
Model #8, MSE: 432.1767534773105
Model #9, MSE: 634.6194547758579
Model #10, MSE: 370.0976272596506


Championship Prediction (Historic)
----------------------------------

1/1 [==============================] - 0s 18ms/step
1994 NBA Season
	Actual Champion: Houston Rockets
	Predicted Champion: Phoenix Suns

1/1 [==============================] - 0s 14ms/step
1995 NBA Season
	Actual Champion: Houston Rockets
	Predicted Champion: Houston Rockets

1/1 [==============================] - 0s 13ms/step
1996 NBA Season
	Actual Champion: Chicago Bulls
	Predicted Champion: Chicago Bulls

1/1 [==============================] - 0s 13ms/step
1997 NBA Season
	Actual Champion: Chicago Bulls
	Predicted Champion: Chicago Bulls

1/1 [=======

## Make Prediction

In [7]:
# Get the 2024 NBA Season Data
df = pd.read_csv(f"./data/2024.csv")

# Get the Model Prediction
X = preprocessor.fit_transform(df[selected_features])
y = best_model.predict(X)
pred_champion = df.iloc[np.argmax(y)]['Team']

# Evaluate Results
print(f"2024 Predicted Champion: {pred_champion}")

# Print Championship Probabilities Per Team
print("\n\nChampionship Probabilities (By Team)\n-------------------------------------")
for i in range(len(y)):
    print(f"{df.iloc[i]['Team']}: {(y[i][0] * 100):.2f}%")

1/1 [==============================] - 0s 13ms/step
2024 Predicted Champion: Boston Celtics


Championship Probabilities (By Team)
-------------------------------------
Atlanta Hawks: 0.03%
Boston Celtics: 48.89%
Brooklyn Nets: 0.09%
Charlotte Hornets: 0.00%
Chicago Bulls: 0.28%
Cleveland Cavaliers: 6.99%
Dallas Mavericks: 9.07%
Denver Nuggets: 7.48%
Detroit Pistons: 0.00%
Golden State Warriors: 18.76%
Houston Rockets: 3.51%
Indiana Pacers: 0.14%
Los Angeles Clippers: 14.03%
Los Angeles Lakers: 0.00%
Memphis Grizzlies: 0.00%
Miami Heat: 10.97%
Milwaukee Bucks: 9.66%
Minnesota Timberwolves: 27.53%
New Orleans Pelicans: 2.98%
New York Knicks: 15.31%
Oklahoma City Thunder: 1.04%
Orlando Magic: 0.70%
Philadelphia 76ers: 1.74%
Phoenix Suns: 20.43%
Portland Trail Blazers: 0.00%
Sacramento Kings: 0.04%
San Antonio Spurs: 0.00%
Toronto Raptors: 0.00%
Utah Jazz: 0.00%
Washington Wizards: 0.00%
